# Basic Setup and Functions

In [1]:
%pip install snntorch --quiet

import librosa, random
import numpy as np
import pandas as pd
import os
import soundfile as sf

from pandas import DataFrame as df
import torch

from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import snntorch as snn
from snntorch.functional.acc import _population_code, _prediction_check

import torch.nn as nn
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torchvision import transforms

from tqdm.notebook import tqdm

import gc

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Note: you may need to restart the kernel to use updated packages.
Using mps device


In [2]:
def Triangle_Network(num_inputs, num_outputs, beta=0.90, time_dependent = False):
    dy_dx = int(4/(num_outputs - num_inputs))
    hidden1 = num_inputs + (dy_dx * 1)
    hidden2 = num_inputs + (dy_dx * 2)
    hidden3 = num_inputs + (dy_dx * 3)

    if beta and time_dependent:
        class Net(nn.Module):
        # Initialise network with 2 forward connections (linear connections) and 2 leaky integrated fire layers (hidden and output)
            def __init__(self, *args, **kwargs) -> None:
                super().__init__(*args, **kwargs)
                self.fc1 = nn.Linear(num_inputs, hidden1)
                self.lif1 = snn.Leaky(beta=beta)
                self.fc2 = nn.Linear(hidden1, hidden2)
                self.lif2 = snn.Leaky(beta=beta)
                self.fc3 = nn.Linear(hidden3, hidden3)
                self.lif3 = snn.Leaky(beta=beta)
                self.fc4 = nn.Linear(hidden3, num_outputs)
                self.lif4 = snn.Leaky(beta=beta)

            # Define a forward pass assuming x is normalised data (i.e. all values in [0,1])
            def forward(self, x):
                mem1 = self.lif1.init_leaky()
                mem2 = self.lif2.init_leaky()
                mem3 = self.lif3.init_leaky()
                mem4 = self.lif4.init_leaky()

                spk_rec = []
                mem_rec = []

                # Insert data in shape (time x batch x features)
                for step in range(x.size(0)):
                    cur1 = self.fc1(x[step])
                    spk1, mem1 = self.lif1(cur1, mem1)
                    cur2 = self.fc2(spk1)
                    spk2, mem2 = self.lif2(cur2, mem2)
                    cur3 = self.fc3(spk2)
                    spk3, mem3 = self.lif3(cur3, mem3)
                    cur4 = self.fc4(spk3)
                    spk4, mem4 = self.lif4(cur4, mem4)

                    spk_rec.append(spk4)
                    mem_rec.append(mem4)

                return torch.stack(spk_rec, dim=0), torch.stack(mem_rec, dim=0)
            
        return Net()


    elif beta and not time_dependent: return nn.Sequential(nn.Flatten(),
                    nn.Linear(num_inputs, hidden1),
                    snn.Leaky(beta=beta, init_hidden=True),
                    nn.Linear(hidden1, hidden2),
                    snn.Leaky(beta=beta, init_hidden=True),
                    nn.Linear(hidden2, hidden3),
                    snn.Leaky(beta=beta, init_hidden=True),
                    nn.Linear(hidden3, num_outputs),
                    snn.Leaky(beta=beta, init_hidden=True, output=True))

    else: return nn.Sequential(nn.Flatten(),
                    nn.Linear(num_inputs, hidden1),
                    nn.ReLU(),
                    nn.Linear(hidden1, hidden2),
                    nn.ReLU(),
                    nn.Linear(hidden2, hidden3),
                    nn.ReLU(),
                    nn.Linear(hidden3, num_outputs))

In [3]:
def test_network(model, dataset, loss_fn, results: df, epoch, device, printable=None, train_test = 'test'):
    dataloader = DataLoader(dataset, batch_size=100, shuffle=False)
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_predicted = []
        all_probs = []

        for data, labels in dataloader:
            x, labels = data.to(device), labels.to(device)
            outputs = model(x)
            test_loss += loss_fn(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predicted.extend(predicted.cpu().numpy())


            softmax = torch.nn.Softmax(dim=1)
            probabilities = softmax(outputs)
            all_probs.extend(probabilities.cpu().numpy())

    # Accuracy
    accuracy = 100 * correct / total

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predicted)

    # Recall/Sensitivity -- avoiding div by 0
    recall = recall_score(all_labels, all_predicted, average='weighted', zero_division=0)

    # Precision
    precision = precision_score(all_labels, all_predicted, average='weighted', zero_division=0)

    # F1 Score
    f1_score = (2 * precision * recall) / (precision + recall)

    # AUC-ROC
    auc_roc = 100 * roc_auc_score(all_labels, all_probs, multi_class='ovr')
    
    if printable: printable.set_description(
        f'Epoch [{epoch + 1}] {train_test} Loss: {test_loss / len(dataloader):.2f} '
        f'{train_test} Accuracy: {accuracy:.2f}% F1: {f1_score}% Recall: {recall:.2f}% Precision: {precision:.2f}% '
        f'AUC-ROC: {auc_roc:.4f}%'
    )

    results = results._append({
            'Epoch': epoch + 1,
            'Accuracy': accuracy,
            'F1': f1_score,
            'Recall': recall,
            'Precision': precision,
            'Test Loss': test_loss / len(dataloader),
            'AUC-ROC': auc_roc,
            'Confusion Matrix': cm
        }, ignore_index=True)

    del data
    del labels
    gc.collect()
    torch.cuda.empty_cache()

    return results


def test_spiking_network(model, dataset, loss_fn, results: df, epoch, device, num_classes=False, printable=None, train_test = 'test'):
    dataloader = DataLoader(dataset, batch_size=100, num_workers=3, shuffle=False)
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_predicted = []
        all_probs = []

        for data, labels in dataloader:
            x, labels = data.transpose(0, 1).to(device), labels.to(device)
            spikes, _ = model(x)
            test_loss += loss_fn(spikes, labels).item()
            
            if num_classes: _, predicted = _population_code(spikes, num_classes=num_classes, num_outputs=spikes.size(-1)).max(1)
            else: _, predicted = spikes.sum(dim=1).max(1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predicted.extend(predicted.cpu().numpy())

            if num_classes: num_spikes = _population_code(spikes, num_classes=num_classes, num_outputs=spikes.size(-1))
            else: num_spikes = spikes.sum(dim=1)
            
            softmax = torch.nn.Softmax(dim=1)
            probabilities = softmax(num_spikes)
            all_probs.extend(probabilities.cpu().numpy())

    # Accuracy
    accuracy = 100 * correct / total

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predicted)

    # Recall/Sensitivity -- avoiding div by 0
    recall = recall_score(all_labels, all_predicted, average='weighted', zero_division=0)

    # Precision
    precision = precision_score(all_labels, all_predicted, average='weighted', zero_division=0)

    # F1 Score
    f1_score = (2 * precision * recall) / (precision + recall)

    # AUC-ROC
    auc_roc = 100 * roc_auc_score(all_labels, all_probs, multi_class='ovr')
    
    if printable: printable.set_description(
        f'Epoch [{epoch + 1}] {train_test} Loss: {test_loss / len(dataloader):.2f} '
        f'{train_test} Accuracy: {accuracy:.2f}% F1: {f1_score}% Recall: {recall:.2f}% Precision: {precision:.2f}% '
        f'AUC-ROC: {auc_roc:.4f}%'
    )

    results = results._append({
            'Epoch': epoch + 1,
            'Accuracy': accuracy,
            'F1': f1_score,
            'Recall': recall,
            'Precision': precision,
            'Test Loss': test_loss / len(dataloader),
            'AUC-ROC': auc_roc,
            'Confusion Matrix': cm
        }, ignore_index=True)

    del data
    del labels
    gc.collect()
    torch.cuda.empty_cache()

    return results

In [4]:
class PopulationCrossEntropyLoss():
    def __init__(self, num_classes=2):
        self.num_classes = num_classes
        self.__name__ = "PopulationCrossEntropyLoss"

    def __call__(self, spk_out, targets):
        loss_fn = nn.CrossEntropyLoss()
        
        _, _, num_outputs = _prediction_check(spk_out)

        spike_count = _population_code(
                spk_out, self.num_classes, num_outputs
            )

        loss = loss_fn(spike_count, targets)

        return loss


In [5]:
LABEL_MAPPINGS = {
    'westernart/classical': 'Classical',
    'indierock/pop': 'Rock',
    'pop/soul/electronica': 'Electronic',
    'electronica': 'Electronic',
    'jazz': 'Jazz',
    'pop/hiphop/rock': 'Pop',
    'rap/hiphop': 'Hiphop',
    'rock': 'Rock',
    'rock/folk': 'Rock',
    'westernart/baroque': 'Classical',
    'electronica/dance': 'Electronic',
    'westernart/romantic': 'Classical',
    'blues': 'Jazz',
    'pop/folk': 'Pop',
    'westernart/romantic/classical': 'Classical',
    'pop/electronica': 'Electronic',
    'latin': 'Jazz',
    'country/folk': 'Country',
    'indierock/folk/pop': 'Rock',
    'jazz/blues': 'Jazz',
    'pop/rap/rock/hiphop': 'Pop',
    'pop/experimental': 'Pop',
    'blues/rock/jazz': 'Jazz',
    'jazz/adventure': 'Jazz',
    'blues/electronica': 'Jazz',
    'jazz/pop/soul': 'Jazz',
    'funk/electronica': 'Electronic',
    'folk/pop': 'Folk',
    'indierock/rock': 'Rock',
    'jazz/electronica': 'Electronic',
    'hiphop': 'Hiphop',
    'funk/rnb/adventure': 'Soul',
    'pop': 'Pop',
    'hiphop/rap': 'Hiphop',
    'pop/gospel': 'Soul',
    'rap/metal/electronica': 'Electronic',
    'pop/rock/folk': 'Rock',
    'pop/electronica/hiphop': 'Pop',
    'metal/rap': 'Hiphop',
    'country': 'Country',
    'rap/metal': 'Hiphop',
    'country/pop': 'Country',
    'folk': 'Folk',
    'pop/rock/dance': 'Pop',
    'dance': 'Electronic',
    'pop/jazz/latin': 'Jazz',
    'pop/jazz': 'Jazz',
    'funk/rnb/electronica': 'Electronic',
    'funk/blues/jazz': 'Jazz',
    'pop/rock/soul': 'Pop',
    'pop/hiphop': 'Pop',
    'blues/funk': 'Jazz',
    'rap/metal/hiphop': 'Hiphop',
    'blues/jazz/adventure': 'Jazz',
    'folk/indierock': 'Folk',
    'adventure': 'Classical',
    'metal/rock': 'Rock',
    'blues/rock/country': 'Jazz',
    'pop/soul/rnb': 'Soul',
    'blues/rock': 'Jazz',
    'blues/rock/indierock': 'Jazz',
    'country/pop/folk': 'Country',
    'country/blues/rock': 'Country',
    'rock/funk/country': 'Rock',
    'pop/rock': 'Rock',
    'pop/blues': 'Rock',
    'blues/indierock': 'Rock',
    'blues/rock/rnb': 'Rock',
    'blues/pop/folk': 'Jazz',
    'pop/funk/adventure': 'Pop',
    'blues/rock/pop': 'Rock',
    'folk/pop/funk': 'Folk'
}


In [6]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rsync -av --exclude='mel_spectrograms' --exclude='cqt_spectrograms' /content/drive/MyDrive/spectrogram_tensors/ /content/spectrogram_tensors/ --quiet
    FILEPATH = '/content'
    CSV = 'sample_ISD.csv'

    dataset = pd.read_csv(f'/content/drive/MyDrive/{CSV}', index_col=0)

    
except:
    FILEPATH = "../../Datasets/SmallDataset"
    ORIGINAL_DIR = "audio"
    SAMPLE_DIR = "audio uncompressed samples"
    COMPRESSED_DIR = "audio compressed"
    CSV = 'sample_ISD.csv'

    dataset = pd.read_csv(f'{FILEPATH}/{CSV}', index_col=0)

X = dataset['filename'].tolist()
Y = dataset[dataset['supercategory']=='music']['category'].map(lambda x: LABEL_MAPPINGS[x]).tolist()
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

label_mappings = {encoded_label: original_label for original_label, encoded_label in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print(label_mappings)

{0: 'Classical', 1: 'Country', 2: 'Electronic', 3: 'Folk', 4: 'Hiphop', 5: 'Jazz', 6: 'Pop', 7: 'Rock', 8: 'Soul'}


In [ ]:
# Garbage collection special commands

gc.collect()
if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.memory_summary(device=None, abbreviated=False)
elif device == "mps":
    torch.mps.empty_cache()
    print(f"MPS occupied memory: {torch.mps.driver_allocated_memory()}")


In [ ]:
# Very special command -- remove all variables
%reset

# Audio Representation

## Sampling

In [ ]:
def sample(audio_path, duration=5.0, sr=44100):
    original_path = audio_path
    for ext in [".m4a", ".wav", ".ogg", ".flac", ".mp3"]:
        if os.path.exists(audio_path + ext):
            audio_path += ext

            total_duration = librosa.get_duration(path=audio_path)
            y, _ = librosa.load(audio_path, sr=sr, duration=total_duration)

            if total_duration < duration:
                pad_length = int((duration - total_duration) * sr)
                y = np.pad(y, (0, pad_length), mode='constant')

            start = random.uniform(0, max(0, total_duration - duration))
            y = y[int(start * sr):int((start + duration) * sr)]

            sf.write(f"{FILEPATH}/{SAMPLE_DIR}/{original_path.split('/')[-1]}.wav", y, sr)

            return y

## Bitrate and Compression

In [ ]:
import os
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

def convert_to_mp3(input_file, output_file, sample_rate=16000, bit_rate="8k", channels=1):
    !ffmpeg -i "$input_file" -ar "$sample_rate" -b:a "$bitrate" -ac "$channels" "$output_file" -hide_banner -loglevel error

def convert_directory_to_mp3(input_dir, output_dir, sample_rate=16000, bit_rate="8k"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    audio_files = [file for file in os.listdir(input_dir) if file.endswith((".m4a", ".wav", ".ogg", ".flac", ".mp3"))]

    for file in tqdm(audio_files, desc="Converting"):
            input_file_path = os.path.join(input_dir, file)
            output_file_path = os.path.join(output_dir, os.path.splitext(file)[0] + ".mp3")
            convert_to_mp3(input_file_path, output_file_path, sample_rate, bit_rate)

In [ ]:
bitdepths = np.array([2,4,8,16,24])
samplerates = np.int32(np.array([8,16,22.05,32,44.1])*1000)

with ThreadPoolExecutor() as executor:
    for bitdepth in bitdepths:
        for samplerate in samplerates:
            bitrate = (bitdepth * samplerate) / 1000
            print(f"bitdepth: {bitdepth}, samplerate: {samplerate}")
            print(f"effective bitrate: {bitrate} kbps")

            executor.submit(convert_directory_to_mp3(f"{FILEPATH}/{SAMPLE_DIR}", f"{FILEPATH}/{COMPRESSED_DIR}/{bitdepth}-{samplerate}", sample_rate=samplerate, bit_rate=f"{bitrate}k"))

In [ ]:
# Assuming the directory contains all compressed files
!find . -mindepth 1 -maxdepth 1 -type d -exec sh -c 'find "$1" -type f -exec ls -l {} \; | awk "{sum += \$5} END {print \"$1\", sum}"' _ {} \

In [ ]:
from mutagen.mp3 import MP3

def get_unpacked_size(mp3_file):
    audio = MP3(mp3_file)
    duration = audio.info.length  # Duration of the audio in seconds
    bitrate = audio.info.bitrate  # Bitrate of the audio in bits per second
    # Calculate the unpacked size based on bitrate and duration
    unpacked_size = (duration * bitrate) / 8
    return unpacked_size


file_dirs = [d for d in os.listdir(f"{FILEPATH}/{COMPRESSED_DIR}") if os.path.isdir(f"{FILEPATH}/{COMPRESSED_DIR}/{d}")]
for bitrate in file_dirs:
    audio_files = [file for file in os.listdir(f"{FILEPATH}/{COMPRESSED_DIR}/{bitrate}") if file.endswith((".mp3"))]
    size = 0
    for file in audio_files:
        if os.path.exists(f"{FILEPATH}/{COMPRESSED_DIR}/{bitrate}/{file}"):
            size += get_unpacked_size(f"{FILEPATH}/{COMPRESSED_DIR}/{bitrate}/{file}")
    print(f"{bitrate.split('/')[-1]}: {size} bytes")


In [ ]:
from matplotlib import pyplot as plt
import tikzplotlib

bit_two = np.array([[16,32,44.1,64,88.2],[1,3,4,5,5]]).T
bit_four = np.array([[32,64,88.2,128,176.4],[2,4,5,5,5]]).T
bit_eight = np.array([[64,128,176.4,256,352.8],[2,4,5,5,5]]).T
bit_sixteen = np.array([[128,256,352.8,512,705.6],[2,3,4,4,5]]).T
bit_twentyfour = np.array([[192,384,529.2,768,1058.4],[2,4,5,5,5]]).T

plt.plot(bit_two[:,0], bit_two[:,1], label="2-bit")
plt.plot(bit_four[:,0], bit_four[:,1], label="4-bit")
plt.plot(bit_eight[:,0], bit_eight[:,1], label="8-bit")
plt.plot(bit_sixteen[:,0], bit_sixteen[:,1], label="16-bit")
plt.plot(bit_twentyfour[:,0], bit_twentyfour[:,1], label="24-bit")

plt.ylabel("Perceived Quality")
plt.xlabel("Bitrate (kbps)")
plt.xscale("log")
plt.xlim(10,1100)
plt.ylim(0, 6)
plt.grid(True, which='both', axis='y')
#plt.legend()

tikzplotlib.save("AudioRep/CompressionReception.tex")

## Spectrograms

In [ ]:
import spectrograms

AUDIO_DIR = "compressed_audio"

X = dataset[dataset['supercategory']=='music']['filename'].tolist()
Y = dataset[dataset['supercategory']=='music']['category'].map(lambda x: LABEL_MAPPINGS[x]).tolist()

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

label_mappings = {encoded_label: original_label for original_label, encoded_label in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print(label_mappings)

X_train, X_test, y_train, y_test = train_test_split(X, Y_encoded, test_size=0.2)

waveforms_train = [spectrograms.load_from_path(f"{FILEPATH}/{AUDIO_DIR}/{file}.mp3") for file in X_train]
waveforms_test = [spectrograms.load_from_path(f"{FILEPATH}/{AUDIO_DIR}/{file}.mp3") for file in X_test]

### Standard Spectrogram

In [ ]:
DIR = "spectrogram_tensors"


for n_fft in [512, 1024, 2048, 4096]:
    for win_length in [512, 1024, 2048, 4096]:
        if n_fft < win_length:
            continue
        else:
            os.makedirs(f"{FILEPATH}/{DIR}/spectrograms/{n_fft}-{512}-{win_length}", exist_ok=True)
            spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.spectrogram(sample, sr, hop=512, fft=n_fft, win=win_length) for sample, sr in waveforms_train]]))
            spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.spectrogram(sample, sr, hop=512, fft=n_fft, win=win_length) for sample, sr in waveforms_test]]))
            spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
            spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
            torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/spectrograms/{n_fft}-{512}-{win_length}/train.pt")
            torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/spectrograms/{n_fft}-{512}-{win_length}/test.pt")


for hop_length in [256, 1024, 2048]:
    os.makedirs(f"{FILEPATH}/{DIR}/spectrograms/{2048}-{hop_length}-{2048}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/spectrograms/{2048}-{hop_length}-{2048}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/spectrograms/{2048}-{hop_length}-{2048}/test.pt")


### Mel Spectrogram

In [ ]:
DIR = "spectrogram_tensors"


for n_fft in [512, 1024, 2048, 4096]:
    for win_length in [512, 1024, 2048, 4096]:
        if n_fft < win_length:
            continue
        else:
            os.makedirs(f"{FILEPATH}/{DIR}/mel_spectrograms/{n_fft}-{512}-{win_length}-{128}", exist_ok=True)
            spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, fft=n_fft, win=win_length) for sample, sr in waveforms_train]]))
            spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, fft=n_fft, win=win_length) for sample, sr in waveforms_test]]))
            spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
            spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
            torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mel_spectrograms/{n_fft}-{512}-{win_length}-{128}/train.pt")
            torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mel_spectrograms/{n_fft}-{512}-{win_length}-{128}/test.pt")


for hop_length in [256, 1024, 2048]:
    os.makedirs(f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{hop_length}-{2048}-{128}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{hop_length}-{2048}-{128}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{hop_length}-{2048}-{128}/test.pt")


for mel_features in [64, 128, 192, 256]:
    os.makedirs(f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{512}-{2048}-{mel_features}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, mel=mel_features) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mel_spectrogram(sample, sr, mel=mel_features) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{512}-{2048}-{mel_features}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mel_spectrograms/{2048}-{512}-{2048}-{mel_features}/test.pt")

### MFCCs

In [ ]:
DIR = "spectrogram_tensors"

for n_fft in [512, 1024, 2048, 4096]:
    for win_length in [512, 1024, 2048, 4096]:
        if n_fft < win_length:
            continue
        else:
            os.makedirs(f"{FILEPATH}/{DIR}/mfcc_spectrograms/{n_fft}-{512}-{win_length}-{128}-{13}", exist_ok=True)
            spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, fft=n_fft, win=win_length) for sample, sr in waveforms_train]]))
            spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, fft=n_fft, win=win_length) for sample, sr in waveforms_test]]))
            spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
            spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
            torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{n_fft}-{512}-{win_length}-{128}-{13}/train.pt")
            torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{n_fft}-{512}-{win_length}-{128}-{13}/test.pt")


for hop_length in [256, 1024, 2048]:
    os.makedirs(f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{hop_length}-{2048}-{128}-{13}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, hop=hop_length, fft=2048, win=2048) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{hop_length}-{2048}-{128}-{13}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{hop_length}-{2048}-{128}-{13}/test.pt")


for mel_features in [64, 128, 192, 256]:
    os.makedirs(f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{mel_features}-{13}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, mel=mel_features) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, mel=mel_features) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{mel_features}-{13}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{mel_features}-{13}/test.pt")

for mfcc_components in [5, 9, 13, 20]:
    os.makedirs(f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{128}-{mfcc_components}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, mfcc_bins=mfcc_components) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.mfcc_spectrogram(sample, sr, mfcc_bins=mfcc_components) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{128}-{mfcc_components}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/mfcc_spectrograms/{2048}-{512}-{2048}-{128}-{mfcc_components}/test.pt")

### CQT

In [ ]:
for hop_length in [256, 512, 1024, 2048]:
    os.makedirs(f"{FILEPATH}/{DIR}/cqt_spectrograms/{hop_length}", exist_ok=True)
    spectrogram_X_train = Tensor(np.array([x for x, _ in [spectrograms.cqt_spectrogram(sample, sr, hop=hop_length) for sample, sr in waveforms_train]]))
    spectrogram_X_test = Tensor(np.array([x for x, _ in [spectrograms.cqt_spectrogram(sample, sr, hop=hop_length) for sample, sr in waveforms_test]]))
    spectrogram_train = TensorDataset(spectrogram_X_train, torch.LongTensor(y_train))
    spectrogram_test = TensorDataset(spectrogram_X_test, torch.LongTensor(y_test))
    torch.save(spectrogram_train, f"{FILEPATH}/{DIR}/cqt_spectrograms/{hop_length}/train.pt")
    torch.save(spectrogram_test, f"{FILEPATH}/{DIR}/cqt_spectrograms/{hop_length}/test.pt")

# ANN Baseline

In [ ]:
def train(model, train_dataset, test_dataset, num_epochs, device):
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=0.01)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_results = df()
    train_results = df()

    epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_loss += loss.item()

        epoch_progress_bar.update(1)

        # Print average loss for the epoch
        test_results = test_network(model, test_dataset, criterion, test_results, epoch, device, printable=(epoch_progress_bar if ((epoch+1) % 15 == 0) else None), test_train = 'test')
        train_results = test_network(model, train_dataset, criterion, test_results, epoch, device, printable=(epoch_progress_bar if ((epoch+1) % 15 == 0) else None), test_train = 'train')

    del model
    del inputs
    del targets
    del optimizer
    del criterion
    del loss
    gc.collect()
    if device == 'cuda': torch.cuda.empty_cache()
    elif device == 'mps': torch.mps.empty_cache()

    print("Training finished!")
    return test_results, train_results

In [ ]:
DIR = "spectrogram_tensors"
FILEPATH = "/content"

X = dataset[dataset['supercategory']=='music']['filename'].tolist()
Y = dataset[dataset['supercategory']=='music']['category'].map(lambda x: LABEL_MAPPINGS[x]).tolist()

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

label_mappings = {encoded_label: original_label for original_label, encoded_label in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print(label_mappings)

In [ ]:
for spectrogram_type in ['mfcc_spectrograms', 'cqt_spectrograms', 'mel_spectrograms', 'spectrograms']:
  for spectrogram_files in os.listdir(f"{FILEPATH}/{DIR}/{spectrogram_type}"):
    if not os.path.isfile(f"{FILEPATH}/{DIR}/{spectrogram_type}/{spectrogram_files}/train.csv") and os.path.isfile(f"{FILEPATH}/{DIR}/{spectrogram_type}/{spectrogram_files}/train.pt"):
      print(f"{spectrogram_type}/{spectrogram_files}")
      train_dataset = torch.load(f"{FILEPATH}/{DIR}/{spectrogram_type}/{spectrogram_files}/train.pt")
      test_dataset = torch.load(f"{FILEPATH}/{DIR}/{spectrogram_type}/{spectrogram_files}/test.pt")

      x_shape = train_dataset[0][0].shape
      scale_factor = min(30000/(x_shape[0] * x_shape[1]), 1)

      transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Resize(tuple(int(dim * scale_factor) for dim in x_shape), antialias=True)
      ])

      train_dataset = [(transform(sample.numpy()), target) for sample, target in train_dataset]
      test_dataset = [(transform(sample.numpy()), target) for sample, target in test_dataset]

      flattened_x_shape = int(x_shape[0]* scale_factor) * int(x_shape[1] * scale_factor)


      model = Triangle_Network(flattened_x_shape, len(label_encoder.classes_), beta=False).to(device)
      num_epochs = 120

      criterion = nn.CrossEntropyLoss()
      #optimizer = optim.SGD(model.parameters(), lr=0.01)
      optimizer = optim.Adam(model.parameters(), lr=0.0005)

      batch_size = 32
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

      test_results = df()
      train_results = df()

      epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

      for epoch in range(num_epochs):
          running_loss = 0.0
          model.train()
          for inputs, targets in train_loader:
              inputs, targets = inputs.to(device), targets.to(device)

              outputs = model(inputs)
              loss = criterion(outputs, targets)

              loss.backward()
              optimizer.step()
              optimizer.zero_grad()

              running_loss += loss.item()

          epoch_progress_bar.update(1)

          # Print average loss for the epoch
          test_results = test_network(model, test_dataset, criterion, test_results, epoch, device, printable=(epoch_progress_bar if ((epoch+1) % 15 == 0) else None), test_train = 'test')
          train_results = test_network(model, train_dataset, criterion, test_results, epoch, device, printable=(epoch_progress_bar if ((epoch+1) % 15 == 0) else None), test_train = 'train')

      del model
      del inputs
      del targets
      del optimizer
      del criterion
      del loss
      gc.collect()
      if device == 'cuda': torch.cuda.empty_cache()
      elif device == 'mps': torch.mps.empty_cache()

      test_results.to_csv(f"/content/drive/MyDrive/spectrogram_tensors/{spectrogram_type}/{spectrogram_files}/test.csv")
      train_results.to_csv(f"/content/drive/MyDrive/spectrogram_tensors/{spectrogram_type}/{spectrogram_files}/train.csv")

In [ ]:
%reset

# Input Encodings

In [7]:
from snntorch import functional as SF

FILEPATH = "../../"
TEST_TYPE = "Models/IST non-JNB results/input_encoding"
SPECTROGRAMS = ['mel_spectrograms', 'mfcc_spectrograms']

## Direct Encoding

In [ ]:
ENCODING_TYPE = "direct_encoding"

In [ ]:
for spectrogram_type in SPECTROGRAMS:
    if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt"):
        print(f"{spectrogram_type}")
        train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt")
        test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.pt")

        # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
        x_shape = train_dataset[0][0].shape

        # Assuming the shape is t x f
        features_shape = x_shape[1]
        POP_ENCODING = 10
        classes = len(label_encoder.classes_)
        output_shape = classes * POP_ENCODING


        model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
        num_epochs = 120

        criterion = SF.mse_count_loss(correct_rate=1.0, incorrect_rate=0.0, population_code=True, num_classes=classes)

        optimizer = optim.Adam(model.parameters(), lr=0.05)

        batch_size = 125
        train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

        test_results = df()
        train_results = df()

        epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)
    
        for epoch in range(num_epochs):
            running_loss = 0.0
            acc = 0
            total = 0
            model.train()
            for inputs, targets in train_loader:
                inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                spikes, _ = model(inputs)

                loss = criterion(spikes, targets)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                running_loss += loss.item()
                total += spikes.size(1)

            epoch_progress_bar.update(1)

            print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
            
            # Print average loss for the epoch
            if ((epoch+1) % 5 == 0):
                to_print = (epoch_progress_bar if ((epoch+1) % 15 == 0) else None)
                test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

        del model
        del inputs
        del targets
        del optimizer
        del criterion
        del loss
        gc.collect()
        if device == 'cuda': torch.cuda.empty_cache()
        elif device == 'mps': torch.mps.empty_cache()

        test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.csv")
        train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv")

## Time Contrast Imports

In [ ]:
from delta import delta

## Direct Time Contrast

In [ ]:
NON_ENCODED = "direct_encoding"
ENCODING_TYPE = "direct_TC_encoding"

for spectrogram_type in ['mfcc_spectrograms', 'mel_spectrograms']:
    original_test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/test.pt")
    original_train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/train.pt")

    direct_TC_test = delta(original_test_dataset, padding = True)
    direct_TC_train = delta(original_train_dataset, padding = True)

    torch.save(direct_TC_test, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.pt")
    torch.save(direct_TC_train, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt")

In [ ]:
for spectrogram_type in SPECTROGRAMS:
    if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt"):
        print(f"{spectrogram_type}")
        train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt")
        test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.pt")

        # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
        x_shape = train_dataset[0][0].shape

        # Assuming the shape is t x f
        features_shape = x_shape[1]
        POP_ENCODING = 10
        classes = len(label_encoder.classes_)
        output_shape = classes * POP_ENCODING


        model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
        num_epochs = 120

        criterion = PopulationCrossEntropyLoss(num_classes=classes)

        optimizer = optim.Adam(model.parameters(), lr=0.001)

        batch_size = 125
        train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

        test_results = df()
        train_results = df()

        epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)
    
        for epoch in range(num_epochs):
            running_loss = 0.0
            acc = 0
            total = 0
            model.train()
            for inputs, targets in train_loader:
                # inputs in form of (time, batch, features)
                inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                spikes, _ = model(inputs)

                loss = criterion(spikes, targets)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                running_loss += loss.item()
                total += spikes.size(0)

            epoch_progress_bar.update(1)

            print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
            
            # Print average loss for the epoch
            if ((epoch+1) % 5 == 0):
                to_print = (epoch_progress_bar if ((epoch+1) % 15 == 0) else None)
                test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

        del model
        del inputs
        del targets
        del optimizer
        del criterion
        del loss
        gc.collect()
        if device == 'cuda': torch.cuda.empty_cache()
        elif device == 'mps': torch.mps.empty_cache()

        test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.csv")
        train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv")

## Time Contrast or Threshold Based

In [ ]:
from delta import delta

NON_ENCODED = "direct_encoding"
ENCODING_TYPE = "time_contrast"

THRESHOLDS = [0.01, 0.025, 0.05, 0.10, 0.20, 0.50]
OFF_SPIKES = [True, False]

for spectrogram_type in SPECTROGRAMS:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            original_test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/test.pt")
            original_train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/train.pt")

            TC_test = delta(original_test_dataset, padding = True, threshold=threshold, off_spike=off_spike, threshold_as_percentage=True)
            TC_train = delta(original_train_dataset, padding = True, threshold=threshold, off_spike=off_spike, threshold_as_percentage=True)

            os.makedirs(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}", exist_ok=True)

            torch.save(TC_test, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.pt")
            torch.save(TC_train, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt")

In [ ]:
for spectrogram_type in SPECTROGRAMS:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt"):
                print(f"{threshold}_{off_spike}_{spectrogram_type}")
                train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt")
                test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.pt")

                # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
                x_shape = train_dataset[0][0].shape

                # Assuming the shape is t x f
                features_shape = x_shape[1]
                POP_ENCODING = 10
                classes = len(label_encoder.classes_)
                output_shape = classes * POP_ENCODING


                model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
                num_epochs = 120

                criterion = PopulationCrossEntropyLoss(num_classes=classes)

                optimizer = optim.Adam(model.parameters(), lr=0.001)

                batch_size = 120
                train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

                test_results = df()
                train_results = df()

                epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

                for epoch in range(num_epochs):
                    running_loss = 0.0
                    total = 0
                    model.train()
                    for inputs, targets in train_loader:
                        # inputs in form of (time, batch, features)
                        inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                        spikes, _ = model(inputs)

                        loss = criterion(spikes, targets)

                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                        running_loss += loss.item()
                        total += spikes.size(0)

                    epoch_progress_bar.update(1)

                    print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
                    
                    # Print average loss for the epoch
                    if ((epoch+1) % 5 == 0):
                        test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                        train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

                del model
                del inputs
                del targets
                del optimizer
                del criterion
                del loss
                gc.collect()
                if device == 'cuda': torch.cuda.empty_cache()
                elif device == 'mps': torch.mps.empty_cache()

                test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.csv")
                train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.csv")

## Cumulative Time Contrast (SF)

In [ ]:
from delta import delta

NON_ENCODED = "direct_encoding"
ENCODING_TYPE = "step_forward"

THRESHOLDS = [0.01, 0.025, 0.05, 0.10, 0.20, 0.50]
OFF_SPIKES = [True, False]

In [ ]:
for spectrogram_type in SPECTROGRAMS:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            original_test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/test.pt")
            original_train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/train.pt")

            TC_test = delta(original_test_dataset, padding = True, threshold=threshold, off_spike=off_spike, threshold_as_percentage=True, cumulative=True)
            TC_train = delta(original_train_dataset, padding = True, threshold=threshold, off_spike=off_spike, threshold_as_percentage=True, cumulative=True)

            os.makedirs(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}", exist_ok=True)

            torch.save(TC_test, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.pt")
            torch.save(TC_train, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt")

In [ ]:
for spectrogram_type in ['mel_spectrograms']:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt"):
                print(f"{threshold}_{off_spike}_{spectrogram_type}")
                train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.pt")
                test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.pt")

                # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
                x_shape = train_dataset[0][0].shape

                # Assuming the shape is t x f
                features_shape = x_shape[1]
                POP_ENCODING = 10
                classes = len(label_encoder.classes_)
                output_shape = classes * POP_ENCODING


                model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
                num_epochs = 120

                criterion = PopulationCrossEntropyLoss(num_classes=classes)

                optimizer = optim.Adam(model.parameters(), lr=0.001)

                batch_size = 120
                train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

                test_results = df()
                train_results = df()

                epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

                for epoch in range(num_epochs):
                    running_loss = 0.0
                    total = 0
                    model.train()
                    for inputs, targets in train_loader:
                        # inputs in form of (time, batch, features)
                        inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                        spikes, _ = model(inputs)

                        loss = criterion(spikes, targets)

                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                        running_loss += loss.item()
                        total += spikes.size(0)

                    epoch_progress_bar.update(1)

                    print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
                    
                    # Print average loss for the epoch
                    if ((epoch+1) % 5 == 0):
                        test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                        train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

                del model
                del inputs
                del targets
                del optimizer
                del criterion
                del loss
                gc.collect()
                if device == 'cuda': torch.cuda.empty_cache()
                elif device == 'mps': torch.mps.empty_cache()

                test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/test.csv")
                train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{threshold}_{off_spike}/{spectrogram_type}/train.csv")

## Moving Window (MW)

## Rate Encoding Imports

In [8]:
from rate import rate, count

## Count Encoding (Whole Spectrogram)

In [9]:
NON_ENCODED = "direct_encoding"
ENCODING_TYPE = "count_encoding"
SUB = "whole"

for spectrogram_type in SPECTROGRAMS:
    for n_count in [5,7,10,15]:
        original_test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/test.pt")
        original_train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/train.pt")

        poisson_test = count(original_test_dataset, max_spikes=n_count)
        poisson_train = count(original_train_dataset, max_spikes=n_count)

        os.makedirs(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}", exist_ok=True)

        torch.save(poisson_test, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/test.pt")
        torch.save(poisson_train, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/train.pt")

In [11]:
for spectrogram_type in ['mfcc_spectrograms']:
    for n_count in [5,7,10,15]:
        if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/train.pt"):
            print(f"{spectrogram_type}")
            train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/train.pt")
            test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/test.pt")

            # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
            x_shape = train_dataset[0][0].shape

            # Assuming the shape is t x f
            features_shape = x_shape[1]
            POP_ENCODING = 10
            classes = len(label_encoder.classes_)
            output_shape = classes * POP_ENCODING


            model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
            num_epochs = 120

            criterion = PopulationCrossEntropyLoss(num_classes=classes)

            optimizer = optim.Adam(model.parameters(), lr=0.001)

            batch_size = 120
            train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

            test_results = df()

            epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

            for epoch in range(num_epochs):
                running_loss = 0.0
                total = 0
                model.train()
                for inputs, targets in train_loader:
                    # inputs in form of (time, batch, features)
                    inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                    spikes, _ = model(inputs)

                    loss = criterion(spikes, targets)

                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                    running_loss += loss.item()
                    total += spikes.size(0)

                epoch_progress_bar.update(1)

                print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
                
                # Print average loss for the epoch
                if ((epoch+1) % 5 == 0):
                    test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                    train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

            del model
            del inputs
            del targets
            del optimizer
            del criterion
            del loss
            gc.collect()
            if device == 'cuda': torch.cuda.empty_cache()
            elif device == 'mps': torch.mps.empty_cache()

            test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/test.csv")
            train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{SUB}/{n_count}/{spectrogram_type}/train.csv")

mfcc_spectrograms


Training Progress:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 1 Running loss: 1.705998533964157
Epoch 2 Running loss: 6.301096343994141
Epoch 3 Running loss: 2.0514718174934385
Epoch 4 Running loss: 2.4342462062835692
Epoch 5 Running loss: 1.6988594651222229
Epoch 6 Running loss: 2.2174144268035887
Epoch 7 Running loss: 1.795849609375


Python(29029) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29030) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29031) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29032) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29034) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29035) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29036) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29037) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(29039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 8 Running loss: 0.9979353904724121


...

Python(32692) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32693) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32694) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32695) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32696) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32697) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32698) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32699) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32700) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32701) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 23 Running loss: 0.24386366009712218


Python(32703) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32704) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32705) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32706) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32707) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32708) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32709) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32710) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32711) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32712) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 24 Running loss: 0.23959487676620483


Python(32716) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32717) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32718) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32719) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32720) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32721) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32723) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32724) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(32725) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Count Encoding (Extending Dims)

## Poisson Encoding (Whole Spectrogram)

## Poisson Encoding (Keeping Dimensions)

In [ ]:
NON_ENCODED = "direct_encoding"
ENCODING_TYPE = "poisson_dims"

for spectrogram_type in SPECTROGRAMS:
    original_test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/test.pt")
    original_train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{NON_ENCODED}/{spectrogram_type}/train.pt")

    poisson_test = rate(original_test_dataset)
    poisson_train = rate(original_train_dataset)

    os.makedirs(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}", exist_ok=True)

    torch.save(poisson_test, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.pt")
    torch.save(poisson_train, f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt")

In [ ]:
for spectrogram_type in SPECTROGRAMS:
    if not os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv") and os.path.isfile(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt"):
        print(f"{spectrogram_type}")
        train_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.pt")
        test_dataset = torch.load(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.pt")

        # Get the shape of the first sample (train_dataset[0]) of data (x) within the dataset
        x_shape = train_dataset[0][0].shape

        # Assuming the shape is t x f
        features_shape = x_shape[1]
        POP_ENCODING = 10
        classes = len(label_encoder.classes_)
        output_shape = classes * POP_ENCODING


        model = Triangle_Network(features_shape, output_shape, beta=0.9, time_dependent=True).to(device)
        num_epochs = 120

        criterion = PopulationCrossEntropyLoss(num_classes=classes)

        optimizer = optim.Adam(model.parameters(), lr=0.001)

        batch_size = 120
        train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)

        test_results = df()

        epoch_progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0)

        for epoch in range(num_epochs):
            running_loss = 0.0
            total = 0
            model.train()
            for inputs, targets in train_loader:
                # inputs in form of (time, batch, features)
                inputs, targets = inputs.transpose(0, 1).to(device), targets.to(device)

                spikes, _ = model(inputs)

                loss = criterion(spikes, targets)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                running_loss += loss.item()
                total += spikes.size(0)

            epoch_progress_bar.update(1)

            print(f"Epoch {epoch+1} Running loss: {running_loss/total}")
            
            # Print average loss for the epoch
            if ((epoch+1) % 5 == 0):
                test_results = test_spiking_network(model, test_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'test')
                train_results = test_spiking_network(model, train_dataset, criterion, test_results, epoch, device, num_classes = classes, printable = epoch_progress_bar, train_test = 'train')

        del model
        del inputs
        del targets
        del optimizer
        del criterion
        del loss
        gc.collect()
        if device == 'cuda': torch.cuda.empty_cache()
        elif device == 'mps': torch.mps.empty_cache()

        test_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/test.csv")
        train_results.to_csv(f"{FILEPATH}/{TEST_TYPE}/{ENCODING_TYPE}/{spectrogram_type}/train.csv")

## Autoencoder

# Architectures & Action Potentials

## MLP (Triangle)

## Convolutional Neural Network